In [ ]:
# 필요한 모듈 불러오기
# FAISS: 대규모 벡터 데이터를 저장하고 유사도를 기반으로 검색할 수 있는 라이브러리
# PyPDFLoader: PDF 파일을 로드하여 텍스트 데이터를 추출하는 모듈
# CharacterTextSplitter: 문서를 여러 청크로 나누기 위한 텍스트 분할기
# OpenAIEmbeddings: 텍스트 데이터를 벡터로 변환하는 OpenAI 임베딩 모델
# ChatOpenAI: OpenAI의 GPT 모델을 사용하여 대화를 처리하는 모듈

In [1]:
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
import os

In [ ]:
# TODO: 위에서 불러온 모듈을 기반으로 빈칸을 완성하세요.

# OpenAI API 키 설정 (실제 API 키로 변경 필요)
os.environ["OPENAI_API_KEY"] = ""

In [3]:
# Step 1: PDF 파일 로드 함수
# PDF 파일을 로드하여 텍스트 데이터를 추출하는 함수입니다.
def load_traffic_data(pdf_file_path):
    loader = PyPDFLoader(pdf_file_path)  # PDF 로더 생성
    documents = loader.load()  # PDF에서 문서(페이지) 불러오기
    print(f"{len(documents)}개의 페이지에서 텍스트를 로드했습니다.")  # 페이지 수 출력
    return documents  # 로드된 문서 반환

In [4]:
# Step 2: 문서를 청크로 나누는 함수
# 문서들을 일정 크기의 청크로 나누어 모델이 처리할 수 있는 단위로 나눕니다.
def split_documents(documents):
    text_splitter = CharacterTextSplitter(
        separator="\n",  # 청크를 분할할 때 사용할 구분자 (줄바꿈 기준)
        chunk_size=300,  # 한 청크의 최대 길이 (단위: 문자 수)
        chunk_overlap=100  # 청크 간 겹치는 부분 길이 (중복된 부분을 포함하여 문맥 유지)
    )
    splits = text_splitter.split_documents(documents)  # 문서를 청크로 나누기
    print(f"{len(splits)}개의 청크로 나누었습니다.")  # 청크 수 출력
    return splits  # 나눈 청크 반환

In [5]:
# Step 3: 청크를 벡터로 변환하여 벡터 스토어에 저장하는 함수
# OpenAI 임베딩을 사용해 청크를 벡터로 변환하고, FAISS 벡터 스토어에 저장합니다.
def store_in_vector_db(splits):
    embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # OpenAI 임베딩 모델 사용
    vector_store = FAISS.from_documents(splits, embeddings)  # 청크를 벡터로 변환하고 저장
    print("벡터 스토어에 청크를 저장했습니다.")  # 저장 완료 메시지 출력
    return vector_store  # 벡터 스토어 반환

In [15]:
# Step 4: 유사한 문서를 검색하는 함수
# 입력한 질문(query)과 가장 유사한 문서를 검색합니다.
def retrieve_similar_docs(query_text, vector_store):
    docs = vector_store.similarity_search(query_text, k=3)  # 유사도 높은 상위 3개의 문서 검색
    print("유사한 문서를 검색했습니다.")  # 검색 완료 메시지 출력
    return docs  # 검색된 문서 반환

In [18]:
# Step 5: LLM을 사용하여 답변 생성하는 함수
# 검색된 문서를 바탕으로 모델이 답변을 생성합니다.
def generate_answer(query_text, docs):
    # OpenAI의 GPT-4 모델을 불러옵니다.
    llm = ChatOpenAI(model_name="gpt-5-nano", temperature=1)
    
    # 시스템 메시지를 통해 모델의 역할을 지정 (교통 전문가로서 답변)
    system_message = SystemMessage(content="너는 교통 전문가야. 질문에 대해 관련된 교통 데이터를 바탕으로 답변해줘.")
    
    # 유저의 질문과 검색된 문서를 담은 메시지 생성
    human_message = HumanMessage(content=f"질문: {query_text}\n\n{docs}")
    
    # 대화에 시스템 메시지와 유저 질문 추가
    conversation = [system_message, human_message]
    
    # 모델에게 대화를 전달하여 답변 생성
    response = llm.invoke(conversation)
    
    return response.content  # 생성된 답변 반환

In [8]:
# Step 6: 예시 사용
# PDF 파일 경로와 검색할 질문을 설정
pdf_file_path = "../data/교통_3대_혁신_전략.pdf"  # PDF 파일 경로 설정
query_text = "교통격차 해소에 투입된 예산은?"  # 검색할 질문 설정

In [11]:
# 1. Loading: PDF 파일 로드
documents = load_traffic_data(pdf_file_path)  # PDF 파일에서 문서 로드

29개의 페이지에서 텍스트를 로드했습니다.


In [12]:
# 2. Splitting: 문서를 청크로 나누기
splits = split_documents(documents)  # 문서를 여러 청크로 분할

Created a chunk of size 715, which is longer than the specified 300
Created a chunk of size 454, which is longer than the specified 300
Created a chunk of size 402, which is longer than the specified 300
Created a chunk of size 316, which is longer than the specified 300
Created a chunk of size 317, which is longer than the specified 300
Created a chunk of size 348, which is longer than the specified 300


70개의 청크로 나누었습니다.


In [13]:
# 3. Storage: 벡터 스토어에 저장
vector_store = store_in_vector_db(splits)  # 청크를 벡터로 변환 후 벡터 스토어에 저장

벡터 스토어에 청크를 저장했습니다.


In [16]:
# 4. Retrieval: 질문에 대한 유사한 문서 검색
similar_docs = retrieve_similar_docs(query_text, vector_store)  # 유사한 문서 검색

유사한 문서를 검색했습니다.


In [19]:
# 5. Generation: 검색된 문서를 바탕으로 답변 생성
answer = generate_answer(query_text, similar_docs)  # 질문에 대한 답변 생성

In [20]:
# 최종 답변 출력
print(f"최종 답변: {answer}")  # 모델이 생성한 답변 출력

최종 답변: 요약 답변
- 교통격차 해소를 포함한 3대 교통혁신 패키지의 재원은 약 134조 원 규모로 집중투자됩니다.
- 구체적 구성은 다음과 같습니다:
  - 국비: 30.0조원
  - 지방비: 13.6조원
  - 민간 재원: 75.2조원
  - 신도시 조성원가 반영: 9.2조원
  - 공공기관 재원: 5.6조원

교통격차 해소와 직결되는 추가 내용
- LH 회계 내 광역교통계정으로 신도시 광역교통개선에 약 11조원 투자 예정.
- GTX-A/B/C 연장 등 광역·도시교통사업은 국비-지방비 매칭으로 추진되며, 보조율은 대체로 아래와 같습니다:
  - 광역철도: 70% (서울은 50%)
  - 도시철도: 60% (서울은 40%)
  - 광역도로: 50%
  - 광역버스 준공영제: 50%
  - BRT: 50%
  - 환승센터: 30%
- 대중교통비 부담 완화를 위한 K-패스 도입(2024년 5월 예정)
  - 환급률: 일반층 20%, 청년층(만19~34세) 30%, 저소득층 53%
  - 혜택 예시: 시내버스·지하철 외에 GTX 등과 연계, 월 이용 60회 기준 환급 월약 1.8만 원(일반층)~4.8만 원(저소득층) 수준
- MaaS 기반 구축 및 교통예보 시스템 도입 등도 포함되어 있어 간접적으로 교통격차 축소에 기여

참고
- 위 내용은 “교통_3대 혁신 전략” 문서의 재원 대책 부분(약 134조원 총투자)과 관련 정책 항목에서 확인됩니다. 더 구체적으로 특정 지역이나 특정 연도별 예산 흐름이 필요하시면 알려주시면 그에 맞춰 자세히 정리해 드리겠습니다.
